# n-Queens in python

In [ ]:
from tkinter import *
import numpy
import random
import time
from threading import Thread
import os
import tkinter.font as TkFont


In [ ]:
window = Tk()
window_width = 750
window_height = 750
board_size = 20
space_width = window_width / board_size
space_height = window_height / board_size


window.wm_geometry(f"{window_width}x{window_height}")
window.title("Chess Board")
window.resizable(False, False)

canvas = Canvas(window, width=window_width, height=window_height)
canvas.pack()

current_best = [0]*board_size
perfect_board = None
final_boards = None
max_children = 100 # Has to be an even value (I just didn't fix it)

In [ ]:
class Board:
    def __init__(self, size=5, square_size_x=50, square_size_y=50):
        self.size = size
        self.square_size_x = square_size_x
        self.square_size_y = square_size_y
        self.color1 = "#DDDDDD"
        self.color2 = "#000000"
        self.letters = [[None for _ in range(self.size)] for _ in range(self.size)]
        self.board = [[None for _ in range(self.size)] for _ in range(self.size)]
        self.create_board()

    def set_board(self, states):
        for i in range(self.size):
            for j in range(self.size):
                if self.letters[j][i] != None:
                    canvas.delete(self.letters[j][i])
        self.letters = [[None for _ in range(self.size)] for _ in range(self.size)]
        for i in range(len(states)):
            self.set_letter(states[i], i, "♕")

    def create_board(self):
        canvas.delete("all")
        for row in range(self.size):
            for col in range(self.size):
                x1 = col * self.square_size_x
                y1 = row * self.square_size_y
                x2 = x1 + self.square_size_x
                y2 = y1 + self.square_size_y
                color = self.color1 if (row + col) % 2 == 0 else self.color2
                cell = canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="")
                self.board[row][col] = cell;
        
    def set_letter(self, row, col, letter):
        if self.letters[row][col] != None:
            canvas.delete(self.letters[row][col])
            del self.letters[row][col]
        x1 = col * self.square_size_x
        y1 = row * self.square_size_y
        x2 = x1 + self.square_size_x
        y2 = y1 + self.square_size_y
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2
        self.letters[row][col] = canvas.create_text(x_center, y_center, font=("Courier", int(44 * space_height / 100)), text=letter,
                                                    fill="#000000" if (row + col) % 2 == 0 else "#FFFFFF")

`fitness_function` is used to determite the score of the board from 0 to 1 (1 being the perfect board without any conflicts)

In [ ]:
def fitness_function(positions):
    conflicts = 0
    n = len(positions)
    for i in range(n):
        for j in range(i + 1, n):
            if positions[i] == positions[j] or \
                    positions[i] + i == positions[j] + j or \
                    positions[i] - i == positions[j] - j:
                conflicts += 1

    fitness_score = 1.0 / (1.0 + conflicts)
    return fitness_score

`get_random_board` returns a list of board positions for every queen on a board. The index functions as a col and the value functions as row (e.g. [4,6,3,7,2,4,5])

In [ ]:
def get_random_board(size):
    return [random.randint(0, size - 1) for _ in range(board_size)]

`evaluate_boards` calls the fitness_function for all the boards in the list

In [ ]:
def evaluate_boards(boards):
    return [fitness_function(i) for i in boards]

`choose_weighted_random_index` returns a random index based on weights (pretty self explanatory)

In [ ]:
def choose_weighted_random_index(numbers):
    total = sum(numbers)
    weighted_numbers = [(number / total) for number in numbers]
    index = random.choices(range(len(numbers)), weights=weighted_numbers)[0]
    return index

`mutate_board` has a chances parts of the board at random

In [ ]:
def mutate_board(_board):
    size = len(_board)
    for i in _board:
        if random.random() < 0.003:
            _board[i] = random.randint(0, size - 1)


`cutter` is a function that gets a list of boards and cuts them in half while fusing them with the next board in the list. 
e.g. `453456` and `367342` make `453342` and `367456`

In [ ]:
def cut(b1,b2,ratio):
    return [b1[i] if i > ratio else b2[i] for i in range(board_size)]

def cutter(_boards):
    children = []
    for i in range(0,len(_boards),2):
        children.append(cut(_boards[i],_boards[i+1],board_size/2))
        children.append(cut(_boards[i+1], _boards[i], board_size/2))
    return children


In [ ]:
def best_board(_boards):
    best = 0
    for i in range(1,len(_boards)):
        if _boards[best] < _boards[i]:
            best = i
    return best


`get_perfect` starts off with a `max_children` amount of random boards and evaluates them. After that a new set of boards are created from the last set's boards (chosen randomly based on their evaluation) with cutting and mutating. 

In [ ]:
def get_perfect():
    global current_best
    global perfect_board
    global final_boards
    all_boards = [get_random_board(board_size) for _ in range(max_children)]
    evaluated = [0]
    best = 0
    survivors = []

    while evaluated[best] != 1:
        del survivors
        del evaluated
        evaluated = evaluate_boards(all_boards)
        best = best_board(evaluated)
        current_best = all_boards[best]
        if evaluated[best] == 1:
            break
        survivors = [all_boards[choose_weighted_random_index(evaluated)] for _ in range(max_children)]
        for i in survivors:
            mutate_board(i)
        all_boards = survivors[:]
        all_boards = cutter(all_boards)
        
       
        

    print("Done",best,all_boards[best],fitness_function(all_boards[best]))
    final_boards = all_boards
    perfect_board = all_boards[best]

In [ ]:
board = Board(size=board_size, square_size_x=space_width, square_size_y=space_height)

`update` periodically prints the best board in the set

In [ ]:
def update():
    board.set_board(current_best)
    print(fitness_function(current_best))
    if fitness_function(current_best) == 1:
        board.set_board(perfect_board)
        return
    window.after(1000,update)


In [ ]:
update()
window_thread = Thread(target=get_perfect, args=())
window_thread.start()
window.mainloop()